# Комментарий:
Предоставляю решение первого задания. При решении второго, столкнулся с проблемой реализации, так как я не смог найти open-source решения на python для разделения на различные голоса. Сложность заключается в том, что тут необходимо взаимодействовать с аудио-волнами, частотами, и если оба говорящих имеют +/- одинаковые голосовые показатели, то задача переходит в более сложную стадию и за 4 дня реализовать такое очень сложно. При анализе источников и решений, я нашел решение у СБЕРа, по разделению на голоса, но доступа к данной технологии у меня нету, так как необходима подписка.
На данный момент у меня имеется решение для распознования речи и перевод ее в текст (V2T), решения по второй части имеется на половину (Без разделения на персоны), но я готов заняться данной задачей в вашей компании (Если необходимо будет внедрить решение с распознованием персон)

#Установка зависимостей

> Добавить блок с цитатой



In [ ]:
!pip install audio-separator
!pip cache purge
!pip install --force-reinstall torch torchvision torchaudio
!pip install --force-reinstall onnxruntime-gpu
!pip install -U git+https://github.com/jianfch/stable-ts.git

# Запись речи с микрофона. Примичание: В Google Collab нет возможности записывать речь с микрофона.

In [ ]:
!pip install pyaudio

In [ ]:
import pyaudio
import wave

chunk = 1024 # Запись кусками по 1024 сэмпла
sample_format = pyaudio.paInt16 # 16 бит на выборку
channels = 2
rate = 44100 # Запись со скоростью 44100 выборок(samples) в секунду
seconds = 3
filename = "audio.wav"
p = pyaudio.PyAudio() # Создать интерфейс для PortAudio

print('Recording...')

stream = p.open(format=sample_format,
channels=channels,
rate=rate,
frames_per_buffer=chunk,
input_device_index=2, # индекс устройства с которого будет идти запись звука
input=True)

frames = [] # Инициализировать массив для хранения кадров

# Хранить данные в блоках в течение 3 секунд
for i in range(0, int(rate / chunk * seconds)):
  data = stream.read(chunk)
  frames.append(data)

# Остановить и закрыть поток
stream.stop_stream()
stream.close()
# Завершить интерфейс PortAudio
p.terminate()

print('Finished recording!')

# Сохранить записанные данные в виде файла WAV
wf = wave.open(filename, 'wb')
wf.setnchannels(channels)
wf.setsampwidth(p.get_sample_size(sample_format))
wf.writeframes(b''.join(frames))
wf.close()

# Модель whisper-3

Выбор данной модели основывался на ее хорошем распозновании текста из голосового файла, также хорошим бонусом является разделение не только на текст, но и на таймкоды.

Можно рассмотреть и другие модели, но данная модель является хорошей в своем сегменте. Еще не мало важным фактором является то, что она имеет исходный открытый код и не требует никаких токенов , по сравнению с аналагами.

Wisper - данную модель имеет несколько размеров параметров, и в зависимости от выбора размерности и будет зависеть качество. Отличным бонусом в данной модели является то, что мы получаем таймкоды, что дает хорошие возможности при обработки аудио файлов.

В пример кода не привожу никакие иные модели, так как они требуют либо токен, либо мощностей для развертывания, которых у меня не хватает. Список моделей, которые были использованы в ходе тестов - pyannote/speaker-diarization, facebook/seamless-m4t-v2-large




In [ ]:
from audio_separator.separator import Separator
from pydub import AudioSegment
import os


#@title Загружаем модель whisper-3

import stable_whisper
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("Устройтво", device)

# Загрузка модели для транскрибации аудиофайлов,
# используя архитектуру "large-3",
# скачивание модели, если она не была предварительно загружена,
# и указание корневой директории для загрузки

if str(device) == "cuda":
    whisper_model = stable_whisper.load_model(
        'large-v3',
        device = "cuda",
        )
else:
    whisper_model = stable_whisper.load_model(
        'large-v3',
        device = device,
        in_memory=True,
        dq=True
        )
#@title Функция для транскребации голоса
import os


def transcribe(input_file):
    # Получаем имя файла без расширения из полного пути к input_file
    input_filename = os.path.splitext(os.path.basename(input_file))[0]

    # Создаем папку для сохранения аудиофайлов, если она не существует
    output_folder = f"{input_filename}"
    os.makedirs(output_folder, exist_ok=True)

    # Выполняем транскрибацию аудиофайла
    # с использованием модели stable_whisper
    result = whisper_model.transcribe(
        os.path.join(output_folder, f"{input_filename}.mp3"),
        )

    # Сохраняем результаты в формате TSV в файл
    # с именем, аналогичным названию входного файла
    result.save_as_json(os.path.join(output_folder, f"{input_filename}.json"))

    return "Done"

Устройтво cpu


100%|█████████████████████████████████████| 2.88G/2.88G [01:46<00:00, 29.0MiB/s]


In [ ]:
!pip install -y ruaccent
!pip install ruaccent

In [ ]:
from transformers import VitsModel, AutoTokenizer, set_seed
import torch
import scipy
from ruaccent import RUAccent

device = 'cuda' #  'cpu' or 'cuda'

speaker = 0 # 0-woman, 1-man

set_seed(555)  # make deterministic

# load model
model_name = "utrobinmv/tts_ru_free_hf_vits_low_multispeaker"

model = VitsModel.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model.eval()

# load accentizer
accentizer = RUAccent()
accentizer.load(omograph_model_size='turbo', use_dictionary=True, device=device)

# text
text = """Я сегодня не приду домой" и отвечает "Ну и катись отсюда"""

# the placement of accents
text = accentizer.process_all(text)
print(text)
# н+очью дв+адцать тр+етьего и+юня н+ачал изверг+аться с+амый выс+окий
# д+ействующий вулк+ан в евр+азии - ключевск+ой. об +этом сообщ+ила
# руковод+итель камч+атской гр+уппы реаг+ирования на вулкан+ические
# изверж+ения, вед+ущий на+учный сотр+удник инстит+ута вулканол+огии
# и сейсмол+огии дво ран +ольга г+ирина. « зафикс+ированное н+очью не
# пр+осто свеч+ение, а верш+инное эксплоз+ивное изверж+ение
# стромболи+анского т+ипа. пок+а так+ое изверж+ение ником+у не оп+асно:
# ни насел+ению, ни ави+ации » поясн+ила тасс госпож+а г+ирина.

inputs = tokenizer(text, return_tensors="pt")

with torch.no_grad():
    output = model(**inputs.to(device), speaker_id=speaker).waveform
    output = output.detach().cpu().numpy()

scipy.io.wavfile.write("tts_audio.wav", rate=model.config.sampling_rate,
                       data=output[0])

# Запускаем процесс транскрибации

In [ ]:
transcribe("audio.wav")

Transcribe:   0%|          | 0/132.34 [00:43<?, ?sec/s]

Detected language: russian


Transcribe: 100%|██████████| 132.34/132.34 [06:10<00:00,  2.80s/sec]

Saved: /content/audio/audio.json


'Done'